## Guider mode quick look
This is intended to unpack the rawStamps from guider mode FITS files and 
give a quick look of all 16 stamps for all 4 CCDs. \
Craig Lage - 12-Apr-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDRC']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = np.array(hdu_list[hduNum].data[0]).astype('>u4')[0]
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

In [ ]:
# Set the scaling
autoscale = False
# Scale to use if autoscale = False
vmin = 14500
vmax = 15000

# Get a quick look at one frame for all 4 CCDs

In [ ]:
dayObs = 20250410

seqNum = 8

raft = 'R00'
ccd = 'SG0'


filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"

rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
print(roiRows, roiCols, roiUnder, nStamps, xor)

In [ ]:
%matplotlib inline

dayObs = 20250410
seqNum = 8
expId = int(f"{dayObs}{seqNum:05d}")
rafts = ['R00', 'R04', 'R40', 'R44']
ccds = ['SG0', 'SG1']
n = 2 # This just unpacks one frame.  This chooses which frame
for raft in rafts[0:1]:
    for ccd in ccds[1:2]:
        filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
        
        rp = ResourcePath(filename)
        with rp.open(mode="rb") as f:
            hdu_list = fits.open(f)
        [roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
        hduNum = 2 * n + 1
        print(hduNum)
        hdrn = hdu_list[hduNum].header
        timestamp = hdrn['STMPTIME']
        image_out = unpackStamps(hduNum)
        fig = plt.figure(figsize=(10,10))
        axs = fig.subplots(4,4)
        plt.subplots_adjust(wspace=0.01, hspace=0.01)
        plt.suptitle(f"Guider mode {raft} {ccd} {expId}, Frame {n+1}\n{timestamp}", fontsize=24)
        for i in range(4):
            for j in range(4):
                seg = i + 4 * j
                if seg == 0:
                    seg0 = image_out[seg]
                #axs[i][j].set_title(f"segment {seg}")
                if autoscale:
                    im = axs[i][j].imshow(image_out[seg], interpolation='nearest', origin='upper', cmap='Greys')
                else:
                    med = np.nanmedian(image_out[seg])
                    vmin = med - 20
                    vmax = med + 20
                    im = axs[i][j].imshow(image_out[seg], interpolation='nearest', origin='upper', vmin=vmin, vmax=vmax, cmap='Greys')
                #divider = make_axes_locatable(axs[i][j])
                #cax = divider.append_axes("right", size="5%", pad=0.05)
                #fig.colorbar(im, cax=cax)
                axs[i][j].set_xticks([])
                axs[i][j].set_yticks([])
                
        #plt.savefig(f"/home/c/cslage/u/Guider_Mode/quick_looks/Guider_QuickLook_{expId}_{raft}_{ccd}.png")

In [ ]:
%matplotlib inline

dayObs = 20250410
seqNum = 8
expId = int(f"{dayObs}{seqNum:05d}")
rafts = ['R00', 'R04', 'R40', 'R44']
ccds = ['SG0', 'SG1']
n = 2 # This just unpacks one frame.  This chooses which frame
for raft in rafts[0:1]:
    for ccd in ccds[1:2]:
        filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
        
        rp = ResourcePath(filename)
        with rp.open(mode="rb") as f:
            hdu_list = fits.open(f)
        [roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
        hduNum = 2 * n + 1
        print(hduNum)
        hdrn = hdu_list[hduNum].header
        timestamp = hdrn['STMPTIME']
        image_out = unpackStamps(hduNum)
        fig = plt.figure(figsize=(12,3))
        axs = fig.subplots(2,8)
        plt.subplots_adjust(wspace=0.01, hspace=0.01)
        plt.suptitle(f"Guider mode {raft} {ccd} {expId}, Frame {n+1}\n{timestamp}", fontsize=24)
        for i in range(2):
            for j in range(8):
                seg = j + 2 * i
                if seg == 0:
                    seg0 = image_out[seg]
                #axs[i][j].set_title(f"segment {seg}")
                if autoscale:
                    im = axs[i][j].imshow(image_out[seg], interpolation='nearest', origin='upper', cmap='Greys')
                else:
                    med = np.nanmedian(image_out[seg])
                    vmin = med - 20
                    vmax = med + 20
                    im = axs[i][j].imshow(image_out[seg], interpolation='nearest', origin='upper', vmin=vmin, vmax=vmax, cmap='Greys')
                #divider = make_axes_locatable(axs[i][j])
                #cax = divider.append_axes("right", size="5%", pad=0.05)
                #fig.colorbar(im, cax=cax)
                axs[i][j].set_xticks([])
                axs[i][j].set_yticks([])
                
        #plt.savefig(f"/home/c/cslage/u/Guider_Mode/quick_looks/Guider_QuickLook_{expId}_{raft}_{ccd}.png")

# Quick look at one of the headers

In [ ]:
hduNum = 0
hdr_test = hdu_list[hduNum].header
for key in hdr_test.keys():
    print(key, hdr_test[key])

In [ ]:
hdr0 = hdu_list[0].header
hdr0['ROIUNDRC']

In [ ]:
image_seg7 = np.array(hdu_list[6].data[0])

In [ ]:
seg7[0] - image_seg7

In [ ]:
image_seg0 = np.array(hdu_list[6].data[0])

In [ ]:
seg0[0] - image_seg0

In [ ]:
data = np.array((hdu_list[5].data[0])).astype('>u4')[0]
data.shape

In [ ]:
data = np.array((hdu_list[5].data[0])).astype('>u4')[0]
data.byteswap(inplace=True)
totalCols = roiCols + roiUnder
size = roiRows * totalCols
out = np.zeros([16, size], dtype=int)
image_out = np.zeros([16, roiRows, roiCols], dtype=int)
print(image_out.shape)
print(roiRows, roiCols, totalCols, size) 
print(len(data))


In [ ]:
fig = plt.figure(figsize=(12,6))
axs = fig.subplots(8,4)

for hduNum in range(2,20,2):
    hdr_test = hdu_list[hduNum].header
    i = (hduNum - 1) % 4
    j = int((hduNum - 1) / 4)
    arr = (hdu_list[hduNum].data)
    axs[j][i].imshow(arr, interpolation='nearest', origin='lower')#, cmap='Greys')

    #print(hduNum, i, j, hdr_test['EXTNAME'])


In [ ]:
for hduNum in range(1,200):
    hdr_test = hdu_list[hduNum].header
    print(hduNum, hdr_test['EXTNAME'])


In [ ]:
np.unravel_index(arr.argmax(), arr.shape)

In [ ]:
arr

In [ ]:
com(arr)